In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [3]:
!mkdir -p project/agents project/core project/memory/sessions project/tools


# Planner

In [4]:
%%writefile project/agents/planner.py
from google import genai
import os
import json

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

class PlannerAgent:
    def __init__(self, memory):
        self.memory = memory

    def plan(self, theme, num_chapters):
        if "outline" in self.memory.data:
            return {"source": "memory", "outline": self.memory.data["outline"]}

        prompt = f"""
You are the Planner Agent.

User theme: {theme}
Number of chapters: {num_chapters}

Create a structured chapter outline in JSON.
If memory is empty, state:
'I am providing this outline from LLM because agents have no memory.'
"""

        r = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt
        )

        outline = r.text
        self.memory.set("outline", outline)

        return {"source": "llm", "outline": outline}


Writing project/agents/planner.py


# Writer


In [5]:


%%writefile project/agents/writer.py
from google import genai
import os

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

class WriterAgent:
    def write_chapter(self, outline, chapter_index):
        prompt = f"""
Write chapter {chapter_index+1} of the story using this outline:

{outline}

Return ONLY the chapter text.
"""

        r = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt
        )

        return r.text


Writing project/agents/writer.py


# Evalutor 

In [6]:
%%writefile project/agents/evaluator.py
from google import genai
import os

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

class EvaluatorAgent:
    def evaluate(self, text):
        prompt = f"""
You are the Evaluator Agent.

Improve grammar, clarity, pacing, and emotional tone.

Fixed version:
{text}
"""

        r = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt,
        )
        return r.text


Writing project/agents/evaluator.py


# Character_consistency

In [7]:
%%writefile project/agents/character_consistency.py
from google import genai
import os

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

class CharacterConsistencyAgent:
    def check(self, character_list, chapter_text):
        prompt = f"""
You are the Character Consistency Agent.
Characters: {character_list}

Check for personality drift or behavior inconsistency.
Return a JSON dict:
- consistent: true/false
- fixed: corrected chapter text
"""

        r = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt
        )
        return r.text


Writing project/agents/character_consistency.py


# continuity_guardian

In [8]:
%%writefile project/agents/continuity_guardian.py
from google import genai
import os

client = genai.Client(api_key=os.getenv("GOOGLE_API_KEY"))

class ContinuityGuardianAgent:
    def check(self, previous_chapters, current_chapter):
        prompt = f"""
You are the Continuity Guardian Agent.

Review the current chapter against previous chapters.
Look for contradictions, plot holes, timeline errors.

Return JSON:
- issues: list
- fixed_chapter: corrected text
"""

        r = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt
        )
        return r.text


Writing project/agents/continuity_guardian.py


# memory

In [9]:
%%writefile project/core/memory.py
import json
import os

class LongMemory:
    def __init__(self, path="project/memory/memory.json"):
        self.path = path
        if not os.path.exists(self.path):
            with open(self.path, "w") as f: json.dump({}, f)
        with open(self.path) as f:
            self.data = json.load(f)

    def set(self, key, value):
        self.data[key] = value
        with open(self.path, "w") as f:
            json.dump(self.data, f, indent=2)

    def get(self, key):
        return self.data.get(key)


Writing project/core/memory.py


# sessions

In [10]:
%%writefile project/core/sessions.py
import os, json, uuid

class SessionManager:
    def __init__(self, base_path="project/memory/sessions"):
        self.base_path = base_path
        os.makedirs(base_path, exist_ok=True)

    def new_session(self):
        sid = str(uuid.uuid4())
        path = f"{self.base_path}/{sid}.json"
        with open(path, "w") as f: json.dump({}, f)
        return sid

    def load(self, sid):
        path = f"{self.base_path}/{sid}.json"
        if not os.path.exists(path):
            with open(path, "w") as f: json.dump({}, f)
        with open(path) as f:
            return json.load(f)

    def save(self, sid, data):
        path = f"{self.base_path}/{sid}.json"
        with open(path, "w") as f: json.dump(data, f, indent=2)


Writing project/core/sessions.py


# Logger

In [11]:
%%writefile project/core/logger.py
import datetime
import os

os.makedirs("project/memory", exist_ok=True)

def log(msg):
    ts = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"[{ts}] {msg}")
    with open("project/memory/logs.txt", "a") as f:
        f.write(f"[{ts}] {msg}\n")


Writing project/core/logger.py


# Streaming

In [12]:
%%writefile project/core/streaming.py
def stream_markdown(text):
    for line in text.split("\n"):
        yield line + "\n"


Writing project/core/streaming.py


# Utils


In [13]:
%%writefile project/tools/utils.py
def clean_text(t):
    return t.strip()


Writing project/tools/utils.py


# coordinator

In [14]:
%%writefile project/core/coordinator.py
from project.agents.planner import PlannerAgent
from project.agents.writer import WriterAgent
from project.agents.evaluator import EvaluatorAgent
from project.agents.character_consistency import CharacterConsistencyAgent
from project.agents.continuity_guardian import ContinuityGuardianAgent

from project.core.logger import log
from project.core.memory import LongMemory

class Coordinator:
    def __init__(self):
        self.memory = LongMemory()
        self.planner = PlannerAgent(self.memory)
        self.writer = WriterAgent()
        self.evaluator = EvaluatorAgent()
        self.char_agent = CharacterConsistencyAgent()
        self.continuity_agent = ContinuityGuardianAgent()

    def create_story(self, theme, num_chapters):
        log("Starting pipeline.")

        plan = self.planner.plan(theme, num_chapters)
        outline = plan["outline"]

        character_list = f"Characters inferred from outline: {outline}"
        self.memory.set("characters", character_list)

        story = []
        for i in range(num_chapters):
            log(f"Writing chapter {i+1}")
            ch = self.writer.write_chapter(outline, i)

            log("Evaluating chapter...")
            improved = self.evaluator.evaluate(ch)

            log("Checking character consistency...")
            consistent = self.char_agent.check(character_list, improved)

            log("Checking continuity...")
            final = self.continuity_agent.check(story, consistent)

            story.append(final)

        self.memory.set("last_story", story)
        log("Story complete.")
        return story


Writing project/core/coordinator.py


# Gradio (App.py)

In [15]:
%%writefile project/app.py
import gradio as gr
from project.core.coordinator import Coordinator
from project.core.streaming import stream_markdown

coordinator = Coordinator()

def generate_story(theme, chapters, session_id):
    chapters = int(chapters)
    story = coordinator.create_story(theme, chapters)

    combined = "\n\n".join([f"### Chapter {i+1}\n{c}" for i,c in enumerate(story)])
    return stream_markdown(combined)

demo = gr.Interface(
    fn=generate_story,
    inputs=[
        gr.Textbox(label="Story Theme"),
        gr.Slider(1, 10, value=3, label="Chapters"),
        gr.Textbox(label="Session ID")
    ],
    outputs=gr.Markdown(),
    title="AI Multi-Agent Story Writer (Gemini)"
)

if __name__ == "__main__":
    demo.launch()


Writing project/app.py


# run_demo

In [16]:
%%writefile project/run_demo.py
from project.core.coordinator import Coordinator

if __name__ == "__main__":
    c = Coordinator()
    story = c.create_story("A misty mountain guarded by shadows", 2)
    print("\n\n".join(story))


Writing project/run_demo.py


# requirements

In [17]:
%%writefile project/requirements.txt
google-genai
gradio


Writing project/requirements.txt


# Test Agents

In [18]:
from project.core.coordinator import Coordinator

c = Coordinator()
story = c.create_story("A crystal valley ruled by ancient spirits", 2)
print(story)


[2025-11-30 17:58:29] Starting pipeline.
[2025-11-30 17:58:35] Writing chapter 1
[2025-11-30 17:58:45] Evaluating chapter...
[2025-11-30 17:58:56] Checking character consistency...
[2025-11-30 17:58:58] Checking continuity...
[2025-11-30 17:59:01] Writing chapter 2
[2025-11-30 17:59:11] Evaluating chapter...
[2025-11-30 17:59:24] Checking character consistency...
[2025-11-30 17:59:28] Checking continuity...
[2025-11-30 17:59:32] Story complete.
['Okay, I\'m ready to assume my role as the Continuity Guardian Agent. Please provide the current chapter text and any relevant information about the previous chapters, plot, and established timeline. The more information you give me, the better I can perform my task. I need the text of the current chapter and at least summaries of the prior chapters, if not the full text, to compare against. Also a summary of the overall plot and important character notes.\n\nOnce I have that, I will:\n\n1.  **Analyze the Current Chapter:**  I\'ll read the chap